[View in Colaboratory](https://colab.research.google.com/github/hackaton-advisors/jupyter/blob/master/shallow_net_in_keras.ipynb)

# Shallow Neural Network in Keras

Build a shallow neural network to classify MNIST digits

#### Set seed for reproducibility

In [41]:
import sys
sys.executable

'/usr/bin/python3'

In [42]:
import numpy as np
np.random.seed(42)

#### Load dependencies

In [43]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD

# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get update -qq 2>&1 > /dev/null
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse


gpg: keybox '/tmp/tmpjoqvz240/pubring.gpg' created
gpg: /tmp/tmpjoqvz240/trustdb.gpg: trustdb created
gpg: key AD5F235DF639B041: public key "Launchpad PPA for Alessandro Strada" imported
gpg: Total number processed: 1
gpg:               imported: 1


#### Load data

In [119]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [120]:
X_train.shape

(60000, 28, 28)

In [46]:
# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()

# Generate creds for the Drive FUSE library.
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
# Work around misordering of STREAM and STDIN in Jupyter.
# https://github.com/jupyter/notebook/issues/3159
prompt = !google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass(prompt[0] + '\n\nEnter verification code: ')
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}


IndexError: ignored

In [5]:
# Create a directory and mount Google Drive using that directory.
!mkdir -p drive
!google-drive-ocamlfuse drive



# Create a file in Drive.
!echo "This newly created file will appear in your Drive file list." > drive/villecreated.txt

Files in Drive:
010c__havainnekuvin.odt
12113A käännös.odt
12113B_no.doc
12113B_no.doc.odt
12452.doc
12452.doc.odt
12847-en.doc.odt
12847 lyhyt.doc
15yr mortgage.ods
1st Grade Class Lists.pdf
2007 Rave Benefits Summary (updated version).odt
2007 Rave Benefits Summary (updated version).odt (3391ee62)
2007 Rave Benefits Summary (updated version).odt (c8c85caf)
2017Resume.odt
446
446.pdf
9592 rakennettavuusselvitys.odt
amtrakaugust 2009 travelexpensereportus1.ods
Annan suostumukseni siihen, että oma ....odt
Annaprasana of AKD4.pdf
backpackthesierra.odt
backpackthesierra.pdf
berkeley-questions.odt
Bicycling directions to Mile High Cafe, California 243, Idyllwild-Pine Cove, CA.desktop
BlackCanyon16
box store -1702 McGaw Ave.desktop
bu.ods
California
capresume.odt
Celebration of Gabe's Life.docx
C-JDBC_Solutions_Linux_2004.pdf
cloudcroft Shared route
Colab Notebooks
colorado
ConciseResume.odt
Configuration management:.odt
Copy of 12113A käännös.odt
Copy of 12113B_no.doc.odt
Copy of capresume

In [10]:
print('Files in Drive:')
!ls drive/

Files in Drive:
010c__havainnekuvin.odt
12113A käännös.odt
12113B_no.doc
12113B_no.doc.odt
12452.doc
12452.doc.odt
12847-en.doc.odt
12847 lyhyt.doc
15yr mortgage.ods
1st Grade Class Lists.pdf
2007 Rave Benefits Summary (updated version).odt
2007 Rave Benefits Summary (updated version).odt (3391ee62)
2007 Rave Benefits Summary (updated version).odt (c8c85caf)
2017Resume.odt
446
446.pdf
9592 rakennettavuusselvitys.odt
amtrakaugust 2009 travelexpensereportus1.ods
Annan suostumukseni siihen, että oma ....odt
Annaprasana of AKD4.pdf
backpackthesierra.odt
backpackthesierra.pdf
berkeley-questions.odt
Bicycling directions to Mile High Cafe, California 243, Idyllwild-Pine Cove, CA.desktop
BlackCanyon16
box store -1702 McGaw Ave.desktop
bu.ods
California
capresume.odt
Celebration of Gabe's Life.docx
C-JDBC_Solutions_Linux_2004.pdf
cloudcroft Shared route
Colab Notebooks
colorado
ConciseResume.odt
Configuration management:.odt
Copy of 12113A käännös.odt
Copy of 12113B_no.doc.odt
Copy of capresume

In [89]:
y_train.shape

(60000,)

In [90]:
y_train[0:99]

array([5, 0, 4, 1, 9, 2, 1, 3, 1, 4, 3, 5, 3, 6, 1, 7, 2, 8, 6, 9, 4, 0,
       9, 1, 1, 2, 4, 3, 2, 7, 3, 8, 6, 9, 0, 5, 6, 0, 7, 6, 1, 8, 7, 9,
       3, 9, 8, 5, 9, 3, 3, 0, 7, 4, 9, 8, 0, 9, 4, 1, 4, 4, 6, 0, 4, 5,
       6, 1, 0, 0, 1, 7, 1, 6, 3, 0, 2, 1, 1, 7, 9, 0, 2, 6, 7, 8, 3, 9,
       0, 4, 6, 7, 4, 6, 8, 0, 7, 8, 3], dtype=uint8)

In [91]:
X_test.shape

(10000, 28, 28)

In [92]:
X_test[1].shape

(28, 28)

In [93]:
X_test[1]

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0, 116, 125, 171,
        255, 255, 150,  93,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0, 169, 253, 253, 253,
        253, 253, 253, 218,  30,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0, 169, 253, 253, 253, 213,
        142, 176, 253, 253, 122,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  

In [0]:
y_test.shape

(10000,)

In [0]:
y_test[1]

2

#### Preprocess data

In [121]:
X_train = X_train.reshape(60000, 784).astype('float32')
X_test = X_test.reshape(10000, 784).astype('float32')

In [122]:
X_train /= 255
X_test /= 255

In [123]:
X_test[0].shape


(784,)

In [124]:
X_test[0]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [125]:
n_classes = 10
y_train = keras.utils.to_categorical(y_train, n_classes)
y_test = keras.utils.to_categorical(y_test, n_classes)

In [126]:
y_test[0]

array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0.], dtype=float32)

#### Design neural network architecture

In [127]:
model = Sequential()
model.add(Dense(64, activation='sigmoid', input_shape=(784,)))
model.add(Dense(10, activation='softmax'))

In [128]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 64)                50240     
_________________________________________________________________
dense_4 (Dense)              (None, 10)                650       
Total params: 50,890
Trainable params: 50,890
Non-trainable params: 0
_________________________________________________________________


In [0]:
(64*784)

50176

In [0]:
(64*784)+64

50240

In [0]:
(10*64)+10

650

#### Configure model

In [129]:
model.compile(loss='mean_squared_error', optimizer=SGD(lr=0.01), metrics=['accuracy'])

#### Train!

In [131]:
model.fit(X_train, y_train, batch_size=128, epochs=200, verbose=1, validation_data=(X_test, y_test))
model.save("drive/numberModel")
#reshapedimg = X_test[5].reshape(1, 784).astype('float32')

#reshapedimg /= 255
result = model.predict(X_test[5],batch_size=1)
result
#resultArray = np.asarray(result[0])
#resultArray.tolist().index(max(result[0]))
#reshapedimg


Train on 60000 samples, validate on 10000 samples
Epoch 1/200
60000/60000 [==============================] - 2s 25us/step - loss: 0.0832 - acc: 0.3938 - val_loss: 0.0829 - val_acc: 0.4003
Epoch 2/200
60000/60000 [==============================] - 2s 29us/step - loss: 0.0828 - acc: 0.3984 - val_loss: 0.0825 - val_acc: 0.4040
Epoch 3/200
60000/60000 [==============================] - 2s 29us/step - loss: 0.0824 - acc: 0.4022 - val_loss: 0.0820 - val_acc: 0.4072
Epoch 4/200
60000/60000 [==============================] - 2s 29us/step - loss: 0.0819 - acc: 0.4060 - val_loss: 0.0816 - val_acc: 0.4103
Epoch 5/200
60000/60000 [==============================] - 2s 30us/step - loss: 0.0815 - acc: 0.4094 - val_loss: 0.0811 - val_acc: 0.4130
Epoch 6/200
60000/60000 [==============================] - 2s 29us/step - loss: 0.0810 - acc: 0.4133 - val_loss: 0.0807 - val_acc: 0.4166
Epoch 7/200
 1920/60000 [..............................]

60000/60000 [==============================] - 2s 29us/step - loss: 0.0806 - acc: 0.4172 - val_loss: 0.0802 - val_acc: 0.4191
Epoch 8/200
60000/60000 [==============================] - 2s 29us/step - loss: 0.0801 - acc: 0.4217 - val_loss: 0.0797 - val_acc: 0.4221
Epoch 9/200
60000/60000 [==============================] - 2s 28us/step - loss: 0.0796 - acc: 0.4257 - val_loss: 0.0792 - val_acc: 0.4270
Epoch 10/200
60000/60000 [==============================] - 2s 28us/step - loss: 0.0791 - acc: 0.4302 - val_loss: 0.0787 - val_acc: 0.4315
Epoch 11/200
60000/60000 [==============================] - 2s 28us/step - loss: 0.0786 - acc: 0.4350 - val_loss: 0.0782 - val_acc: 0.4376
Epoch 12/200
60000/60000 [==============================] - 2s 27us/step - loss: 0.0781 - acc: 0.4406 - val_loss: 0.0777 - val_acc: 0.4429
Epoch 13/200
35200/60000 [================>.............] - ETA: 0s - loss: 0.0779 - acc: 0.4406

60000/60000 [==============================] - 2s 28us/step - loss: 0.0776 - acc: 0.4449 - val_loss: 0.0772 - val_acc: 0.4471
Epoch 14/200
60000/60000 [==============================] - 2s 27us/step - loss: 0.0771 - acc: 0.4496 - val_loss: 0.0767 - val_acc: 0.4511
Epoch 15/200
60000/60000 [==============================] - 2s 27us/step - loss: 0.0766 - acc: 0.4535 - val_loss: 0.0762 - val_acc: 0.4565
Epoch 16/200
60000/60000 [==============================] - 2s 27us/step - loss: 0.0761 - acc: 0.4582 - val_loss: 0.0757 - val_acc: 0.4623
Epoch 17/200
60000/60000 [==============================] - 2s 27us/step - loss: 0.0756 - acc: 0.4635 - val_loss: 0.0752 - val_acc: 0.4662
Epoch 18/200
60000/60000 [==============================] - 2s 27us/step - loss: 0.0751 - acc: 0.4691 - val_loss: 0.0746 - val_acc: 0.4721
Epoch 19/200
41344/60000 [===================>..........] - ETA: 0s - loss: 0.0747 - acc: 0.4721

60000/60000 [==============================] - 2s 28us/step - loss: 0.0746 - acc: 0.4736 - val_loss: 0.0741 - val_acc: 0.4763
Epoch 20/200
60000/60000 [==============================] - 2s 28us/step - loss: 0.0741 - acc: 0.4787 - val_loss: 0.0736 - val_acc: 0.4817
Epoch 21/200
60000/60000 [==============================] - 2s 27us/step - loss: 0.0735 - acc: 0.4850 - val_loss: 0.0731 - val_acc: 0.4868
Epoch 22/200
60000/60000 [==============================] - 2s 28us/step - loss: 0.0730 - acc: 0.4892 - val_loss: 0.0726 - val_acc: 0.4913
Epoch 23/200
60000/60000 [==============================] - 2s 29us/step - loss: 0.0725 - acc: 0.4941 - val_loss: 0.0720 - val_acc: 0.4964
Epoch 24/200
60000/60000 [==============================] - 2s 28us/step - loss: 0.0720 - acc: 0.4997 - val_loss: 0.0715 - val_acc: 0.5022
Epoch 25/200
34176/60000 [================>.............] - ETA: 0s - loss: 0.0715 - acc: 0.5063

60000/60000 [==============================] - 2s 29us/step - loss: 0.0715 - acc: 0.5042 - val_loss: 0.0710 - val_acc: 0.5075
Epoch 26/200
60000/60000 [==============================] - 2s 29us/step - loss: 0.0710 - acc: 0.5097 - val_loss: 0.0705 - val_acc: 0.5125
Epoch 27/200
60000/60000 [==============================] - 2s 28us/step - loss: 0.0705 - acc: 0.5150 - val_loss: 0.0700 - val_acc: 0.5174
Epoch 28/200
60000/60000 [==============================] - 2s 26us/step - loss: 0.0700 - acc: 0.5193 - val_loss: 0.0694 - val_acc: 0.5224
Epoch 29/200
60000/60000 [==============================] - 2s 28us/step - loss: 0.0695 - acc: 0.5244 - val_loss: 0.0689 - val_acc: 0.5260
Epoch 30/200
60000/60000 [==============================] - 2s 29us/step - loss: 0.0689 - acc: 0.5288 - val_loss: 0.0684 - val_acc: 0.5307
Epoch 31/200
36480/60000 [=================>............] - ETA: 0s - loss: 0.0685 - acc: 0.5347

...

60000/60000 [==============================] - 2s 26us/step - loss: 0.0294 - acc: 0.8526 - val_loss: 0.0285 - val_acc: 0.8639
Epoch 182/200
60000/60000 [==============================] - 2s 26us/step - loss: 0.0293 - acc: 0.8528 - val_loss: 0.0284 - val_acc: 0.8639
Epoch 183/200
60000/60000 [==============================] - 2s 26us/step - loss: 0.0292 - acc: 0.8535 - val_loss: 0.0283 - val_acc: 0.8645
Epoch 184/200
60000/60000 [==============================] - 2s 26us/step - loss: 0.0291 - acc: 0.8540 - val_loss: 0.0282 - val_acc: 0.8646
Epoch 185/200
60000/60000 [==============================] - 2s 27us/step - loss: 0.0290 - acc: 0.8544 - val_loss: 0.0280 - val_acc: 0.8648
Epoch 186/200
60000/60000 [==============================] - 2s 27us/step - loss: 0.0288 - acc: 0.8549 - val_loss: 0.0279 - val_acc: 0.8651
Epoch 187/200
48640/60000 [=======================>......] - ETA: 0s - loss: 0.0287 - acc: 0.8553

60000/60000 [==============================] - 2s 26us/step - loss: 0.0287 - acc: 0.8554 - val_loss: 0.0278 - val_acc: 0.8652
Epoch 188/200
60000/60000 [==============================] - 2s 27us/step - loss: 0.0286 - acc: 0.8560 - val_loss: 0.0277 - val_acc: 0.8657
Epoch 189/200
60000/60000 [==============================] - 2s 27us/step - loss: 0.0285 - acc: 0.8563 - val_loss: 0.0276 - val_acc: 0.8660
Epoch 190/200
60000/60000 [==============================] - 2s 27us/step - loss: 0.0284 - acc: 0.8566 - val_loss: 0.0275 - val_acc: 0.8665
Epoch 191/200
60000/60000 [==============================] - 2s 28us/step - loss: 0.0283 - acc: 0.8572 - val_loss: 0.0274 - val_acc: 0.8670
Epoch 192/200
60000/60000 [==============================] - 2s 28us/step - loss: 0.0282 - acc: 0.8576 - val_loss: 0.0273 - val_acc: 0.8669
Epoch 193/200
40704/60000 [===================>..........] - ETA: 0s - loss: 0.0282 - acc: 0.8569

60000/60000 [==============================] - 2s 27us/step - loss: 0.0281 - acc: 0.8580 - val_loss: 0.0272 - val_acc: 0.8674
Epoch 194/200
60000/60000 [==============================] - 2s 26us/step - loss: 0.0280 - acc: 0.8584 - val_loss: 0.0271 - val_acc: 0.8675
Epoch 195/200
60000/60000 [==============================] - 2s 26us/step - loss: 0.0279 - acc: 0.8586 - val_loss: 0.0270 - val_acc: 0.8681
Epoch 196/200
60000/60000 [==============================] - 2s 26us/step - loss: 0.0278 - acc: 0.8591 - val_loss: 0.0269 - val_acc: 0.8687
Epoch 197/200
60000/60000 [==============================] - 2s 27us/step - loss: 0.0277 - acc: 0.8594 - val_loss: 0.0268 - val_acc: 0.8691
Epoch 198/200
60000/60000 [==============================] - 2s 27us/step - loss: 0.0276 - acc: 0.8596 - val_loss: 0.0267 - val_acc: 0.8696
Epoch 199/200
45056/60000 [=====================>........] - ETA: 0s - loss: 0.0277 - acc: 0.8586

60000/60000 [==============================] - 2s 27us/step - loss: 0.0275 - acc: 0.8601 - val_loss: 0.0266 - val_acc: 0.8699
Epoch 200/200
60000/60000 [==============================] - 2s 27us/step - loss: 0.0274 - acc: 0.8604 - val_loss: 0.0265 - val_acc: 0.8701


ValueError: ignored

In [151]:
model = keras.models.load_model("drive/numberModel")

In [145]:
X_test[5].shape

(784,)

In [153]:
model = keras.models.load_model("drive/numberModel")
test_item = X_test[5].reshape(1, 784).astype('float32')
result = model.predict(test_item)
result

array([[0.00098566, 0.88181424, 0.01852677, 0.02414856, 0.00332271,
        0.01339199, 0.00313626, 0.01338676, 0.02982836, 0.01145862]],
      dtype=float32)

In [115]:
model.evaluate(X_test, y_test)

RuntimeError: ignored

In [7]:
import cv2

In [17]:
#test8 = cv2.imread("number8.jpg")
test8 = cv2.imread("drive/number8.jpg", cv2.cv2.IMREAD_GRAYSCALE)
print(test8)

[[162 163 163 ... 164 164 167]
 [159 161 163 ... 165 164 166]
 [160 160 162 ... 165 163 165]
 ...
 [134 134 133 ... 126 126 128]
 [133 133 131 ... 127 127 129]
 [131 130 129 ... 128 128 130]]


In [15]:
!pwd

/content


In [18]:
test8.size

455040

In [0]:
test2[1][2]
#test2

255

In [0]:
test2.shape

(28, 28)

In [0]:
resized_image_test2.size

784

In [23]:
resized_image= cv2.resize(test8, (28, 28)) 

In [24]:
resized_image.shape

(28, 28)

In [25]:
resized_image

array([[166, 160, 162, 159, 160, 160, 160, 162, 164, 162, 164, 161, 164,
        160, 158, 156, 160, 162, 164, 165, 164, 166, 162, 165, 165, 162,
        167, 166],
       [162, 161, 160, 160, 158, 158, 158, 160, 163, 162, 163, 160, 160,
        162, 157, 159, 162, 165, 162, 163, 164, 165, 166, 168, 159, 164,
        163, 163],
       [162, 161, 159, 157, 160, 160, 162, 164, 166, 164, 165, 162, 164,
        164, 163, 162, 162, 162, 158, 159, 152, 169, 162, 161, 160, 164,
        163, 165],
       [160, 158, 157, 159, 159, 156, 161, 161, 163, 161, 161, 161, 162,
        163, 161, 165, 163, 164, 160,  89, 163, 161, 163, 161, 160, 163,
        164, 165],
       [156, 157, 157, 159, 157, 159, 160, 163, 161, 162, 158, 163, 162,
        157, 162, 163, 160, 164, 136, 121, 162, 156, 165, 165, 161, 160,
        165, 163],
       [157, 156, 158, 159, 158, 160, 161, 161, 163, 161, 162, 164, 163,
        158, 166, 157, 146, 145,  82, 163, 167, 159, 164, 164, 160, 163,
        165, 165],
       [16

In [26]:
resized_image[15][16]

67

In [27]:
28*28

784

In [0]:
#newimage = np.resize(x, [28,28])

In [0]:
#newimage

In [34]:
def myfunc(z):
    return abs(z-255)
#a = np.zeros(shape=(28,28))

In [0]:
a.shape

(28, 28)

In [0]:
a

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0

In [39]:
#resized_image = resized_image.astype(np.int16)
#import scipy.misc
face = scipy.misc.face()
def myfunc(z):
    return abs(z-255)
myfunc = np.vectorize(myfunc, otypes=[np.float])
newA = face.astype(np.int16)
myfunc_vec = np.vectorize(myfunc)
result = myfunc_vec(resized_image)
#result = myfunc_vec(resized_image_test2)
result.size


784

In [51]:
result

array([[ 89.,  95.,  93.,  96.,  95.,  95.,  95.,  93.,  91.,  93.,  91.,
         94.,  91.,  95.,  97.,  99.,  95.,  93.,  91.,  90.,  91.,  89.,
         93.,  90.,  90.,  93.,  88.,  89.],
       [ 93.,  94.,  95.,  95.,  97.,  97.,  97.,  95.,  92.,  93.,  92.,
         95.,  95.,  93.,  98.,  96.,  93.,  90.,  93.,  92.,  91.,  90.,
         89.,  87.,  96.,  91.,  92.,  92.],
       [ 93.,  94.,  96.,  98.,  95.,  95.,  93.,  91.,  89.,  91.,  90.,
         93.,  91.,  91.,  92.,  93.,  93.,  93.,  97.,  96., 103.,  86.,
         93.,  94.,  95.,  91.,  92.,  90.],
       [ 95.,  97.,  98.,  96.,  96.,  99.,  94.,  94.,  92.,  94.,  94.,
         94.,  93.,  92.,  94.,  90.,  92.,  91.,  95., 166.,  92.,  94.,
         92.,  94.,  95.,  92.,  91.,  90.],
       [ 99.,  98.,  98.,  96.,  98.,  96.,  95.,  92.,  94.,  93.,  97.,
         92.,  93.,  98.,  93.,  92.,  95.,  91., 119., 134.,  93.,  99.,
         90.,  90.,  94.,  95.,  90.,  92.],
       [ 98.,  99.,  97.,  96.,  97

In [50]:
result.shape
reshapedimg = result.reshape(1, 784).astype('float32')
reshapedimg
#model.predict(reshapedimg)


array([[ 89.,  95.,  93.,  96.,  95.,  95.,  95.,  93.,  91.,  93.,  91.,
         94.,  91.,  95.,  97.,  99.,  95.,  93.,  91.,  90.,  91.,  89.,
         93.,  90.,  90.,  93.,  88.,  89.,  93.,  94.,  95.,  95.,  97.,
         97.,  97.,  95.,  92.,  93.,  92.,  95.,  95.,  93.,  98.,  96.,
         93.,  90.,  93.,  92.,  91.,  90.,  89.,  87.,  96.,  91.,  92.,
         92.,  93.,  94.,  96.,  98.,  95.,  95.,  93.,  91.,  89.,  91.,
         90.,  93.,  91.,  91.,  92.,  93.,  93.,  93.,  97.,  96., 103.,
         86.,  93.,  94.,  95.,  91.,  92.,  90.,  95.,  97.,  98.,  96.,
         96.,  99.,  94.,  94.,  92.,  94.,  94.,  94.,  93.,  92.,  94.,
         90.,  92.,  91.,  95., 166.,  92.,  94.,  92.,  94.,  95.,  92.,
         91.,  90.,  99.,  98.,  98.,  96.,  98.,  96.,  95.,  92.,  94.,
         93.,  97.,  92.,  93.,  98.,  93.,  92.,  95.,  91., 119., 134.,
         93.,  99.,  90.,  90.,  94.,  95.,  90.,  92.,  98.,  99.,  97.,
         96.,  97.,  95.,  94.,  94., 

In [28]:
test8 = cv2.imread("drive/number8.jpg", cv2.cv2.IMREAD_GRAYSCALE)
test8 = cv2.resize(test8, (28, 28)) 
test8.shape
test8

array([[166, 160, 162, 159, 160, 160, 160, 162, 164, 162, 164, 161, 164,
        160, 158, 156, 160, 162, 164, 165, 164, 166, 162, 165, 165, 162,
        167, 166],
       [162, 161, 160, 160, 158, 158, 158, 160, 163, 162, 163, 160, 160,
        162, 157, 159, 162, 165, 162, 163, 164, 165, 166, 168, 159, 164,
        163, 163],
       [162, 161, 159, 157, 160, 160, 162, 164, 166, 164, 165, 162, 164,
        164, 163, 162, 162, 162, 158, 159, 152, 169, 162, 161, 160, 164,
        163, 165],
       [160, 158, 157, 159, 159, 156, 161, 161, 163, 161, 161, 161, 162,
        163, 161, 165, 163, 164, 160,  89, 163, 161, 163, 161, 160, 163,
        164, 165],
       [156, 157, 157, 159, 157, 159, 160, 163, 161, 162, 158, 163, 162,
        157, 162, 163, 160, 164, 136, 121, 162, 156, 165, 165, 161, 160,
        165, 163],
       [157, 156, 158, 159, 158, 160, 161, 161, 163, 161, 162, 164, 163,
        158, 166, 157, 146, 145,  82, 163, 167, 159, 164, 164, 160, 163,
        165, 165],
       [16

In [52]:
result = myfunc_vec(test8)
reshapedimg = result.reshape(1, 784).astype('float32')
reshapedimg /= 255

In [0]:
reshapedimg

array([[0.14901961, 0.14117648, 0.14901961, 0.14117648, 0.13333334,
        0.14117648, 0.12941177, 0.14117648, 0.13725491, 0.14509805,
        0.14901961, 0.14901961, 0.14509805, 0.14117648, 0.14901961,
        0.13333334, 0.13333334, 0.12941177, 0.13333334, 0.13333334,
        0.1254902 , 0.13725491, 0.15294118, 0.12941177, 0.14509805,
        0.14901961, 0.14509805, 0.14117648, 0.14901961, 0.14509805,
        0.13725491, 0.15686275, 0.14117648, 0.13333334, 0.12941177,
        0.12941177, 0.14509805, 0.14509805, 0.15294118, 0.14901961,
        0.12941177, 0.13333334, 0.14117648, 0.14117648, 0.15294118,
        0.13333334, 0.14509805, 0.12941177, 0.13725491, 0.14901961,
        0.13333334, 0.14509805, 0.15294118, 0.16470589, 0.14509805,
        0.14901961, 0.14117648, 0.15294118, 0.14901961, 0.14117648,
        0.14509805, 0.14117648, 0.13725491, 0.14117648, 0.14117648,
        0.14509805, 0.15686275, 0.14117648, 0.13333334, 0.13725491,
        0.14117648, 0.12156863, 0.14509805, 0.14

In [0]:
def invert(z):
    return abs(z-255)

def brighten(z):
    if(z<100):
        return 0.0
    else:
        return z
#a = np.zeros(shape=(28,28))

In [0]:
#test4 = cv2.imread("test4.jpg", cv2.cv2.IMREAD_GRAYSCALE)
#test4 = cv2.resize(test4, (28, 28)) 
#invert = np.vectorize(invert, otypes=[np.float])
#test4 = invert(test4)
#brighten = np.vectorize(brighten, otypes=[np.float])
#test4 = brighten(test4)
reshapedimg = test4.reshape(1, 784).astype('float32')
#
reshapedimg /= 255
reshapedimg
result = model.predict(reshapedimg)
#n_classes = 10
#keras.utils.to_categorical(result, n_classes)

array([[0.00392157, 0.00392157, 0.00392157, 0.00392157, 0.00392157,
        0.00392157, 0.00392157, 0.00392157, 0.00392157, 0.00392157,
        0.00392157, 0.00392157, 0.00392157, 0.00392157, 0.00392157,
        0.00392157, 0.00392157, 0.00392157, 0.00392157, 0.00392157,
        0.00392157, 0.00392157, 0.00392157, 0.00392157, 0.00392157,
        0.00392157, 0.00392157, 0.00392157, 0.00392157, 0.00392157,
        0.00392157, 0.00392157, 0.00392157, 0.00392157, 0.00392157,
        0.00392157, 0.00392157, 0.00392157, 0.00392157, 0.00392157,
        0.00392157, 0.00392157, 0.00392157, 0.00392157, 0.00392157,
        0.00392157, 0.00392157, 0.00392157, 0.00173779, 0.00195309,
        0.00213764, 0.00392157, 0.00392157, 0.00392157, 0.00392157,
        0.00392157, 0.00392157, 0.00392157, 0.00392157, 0.00392157,
        0.00229143, 0.0021684 , 0.00236832, 0.00392157, 0.00392157,
        0.00392157, 0.00392157, 0.00392157, 0.00392157, 0.00392157,
        0.00392157, 0.00392157, 0.00392157, 0.00

In [58]:
result = model.predict(reshapedimg)
result

array([[0.11504813, 0.10483529, 0.08095919, 0.03392908, 0.15166551,
        0.19713983, 0.03848398, 0.0961139 , 0.08815534, 0.09366971]],
      dtype=float32)

In [56]:
def invert(z):
    return abs(z-255)
invert = np.vectorize(invert, otypes=[np.float])
test8 = invert(test8)
cv2.imwrite("/Drive/testsetnum.bmp",test8)

False

In [0]:
#X_test[1]
#test4 = test4.astype(np.uint8)
#test4
reshapedimg = test4.reshape(1, 784).astype('float32')
reshapedimg /= 255
reshapedimg
result = model.predict(reshapedimg)
result
#n_classes = 10
#keras.utils.to_categorical(result, n_classes)

#test4 = invert(test4)
#test4

array([[0.12673727, 0.01990256, 0.08104635, 0.08354645, 0.20915388,
        0.2006497 , 0.05133255, 0.08735745, 0.05018364, 0.09009027]],
      dtype=float32)

In [0]:
resultArray = np.asarray(result[0])
resultArray.tolist().index(max(result[0]))

4

In [0]:
test4 = test4.astype(np.uint8)
test4

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0, 142, 128, 116,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0, 106, 114, 101,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0, 102, 141, 159, 125, 107,   0,   0,
          0,   0],
       [  0,   0,   0,   0, 108, 142, 156, 113,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0, 151, 165, 137, 112,   0,   0,
          0,   0],
       [  0,   0,   0,   0, 113, 193, 156,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0, 107, 171, 183, 167, 119,   0,   0,
          0,   0],
       [  0,   0,   0,   0, 129, 193, 174,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0, 113, 183, 188, 144,   0,   0,   0,
          0,   0],
       [  

In [0]:
test4 = cv2.imread("test4.jpg", cv2.cv2.IMREAD_GRAYSCALE)
test4 = cv2.resize(test4, (28, 28)) 
invert = np.vectorize(invert, otypes=[np.float])
test4 = invert(test4)
brighten = np.vectorize(brighten, otypes=[np.float])
test4 = brighten(test4)
reshapedimg = test4.reshape(1, 784).astype('float32')
reshapedimg /= 255
result = model.predict(reshapedimg)
resultArray = np.asarray(result[0])
resultArray.tolist().index(max(result[0]))

4

In [0]:
def invert(z):
    return abs(z-255)
invert_vec = np.vectorize(invert, otypes=[np.float])
def brighten(z):
    if(z<130):
        return 0.0
    if(z>150):
        return 255.0
    else:
        return z
test8 = cv2.imread("test2.jpg", cv2.cv2.IMREAD_GRAYSCALE)
test8 = cv2.resize(test8, (28, 28)) 
invert = np.vectorize(invert, otypes=[np.float])
test8 = invert(test8)
brighten = np.vectorize(brighten, otypes=[np.float])
test8 = brighten(test8)
test8 = X_test[3]
#test8=test8.astype(np.uint8)
cv2.imwrite("/Temp/testsetnum.bmp",test8)
reshapedimg = test8.reshape(1, 784).astype('float32')
reshapedimg /= 255
result = model.predict(reshapedimg)
resultArray = np.asarray(result[0])
resultArray.tolist().index(max(result[0]))
#result

0

In [0]:
test8

array([[  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 166.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 134.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0

In [88]:
#X_test[1]   cv2.imwrite("drive/testsetnum2.bmp",X_test[1])
reshapedimg = X_test[1].reshape(1, 784).astype('float32')

reshapedimg /= 255
result = model.predict(reshapedimg)

#result = model.predict(X_test[1])
result

array([[0.09048344, 0.11299986, 0.06805648, 0.0363975 , 0.17078878,
        0.21243227, 0.03925431, 0.11154913, 0.08747695, 0.07056123]],
      dtype=float32)

In [0]:
test8 = cv2.imread("test8.jpg", cv2.cv2.IMREAD_GRAYSCALE)

In [0]:
test8 = cv2.resize(test4, (28, 28)) 

In [87]:
cv2.imwrite("drive/testsetnum2.bmp",X_test[1])

True

In [0]:
invert = np.vectorize(invert, otypes=[np.float])

In [0]:
test8 = invert(test8)

In [0]:
brighten = np.vectorize(brighten, otypes=[np.float])
test8 = brighten(test8)

In [0]:
reshapedimg = test8.reshape(1, 784).astype('float32')


In [0]:
reshapedimg /= 255
result = model.predict(reshapedimg)

In [0]:
result

array([[0.20979929, 0.00931124, 0.11923025, 0.10231639, 0.02885046,
        0.221767  , 0.03390548, 0.04421921, 0.15973018, 0.07087056]],
      dtype=float32)

**STARTING FRESH GOOGLE DRIVE NUMBER 8 TEST**

In [162]:
test8 = cv2.imread("drive/number8.jpg", cv2.cv2.IMREAD_GRAYSCALE)
#test8 = cv2.resize(test8, (28, 28)) 
#test8.shape
test8

array([[162, 163, 163, ..., 164, 164, 167],
       [159, 161, 163, ..., 165, 164, 166],
       [160, 160, 162, ..., 165, 163, 165],
       ...,
       [134, 134, 133, ..., 126, 126, 128],
       [133, 133, 131, ..., 127, 127, 129],
       [131, 130, 129, ..., 128, 128, 130]], dtype=uint8)

In [74]:
test8

array([[255., 255., 255., ..., 255., 255., 255.],
       [255., 255., 255., ..., 255., 255., 255.],
       [255., 255., 255., ..., 255., 255., 255.],
       ...,
       [134., 134., 133., ..., 126., 126., 128.],
       [133., 133., 131., ..., 127., 127., 129.],
       [131., 130., 129., ..., 128., 128., 130.]])

In [61]:
cv2.imwrite("drive/initialRead.bmp",test8)

True

In [163]:
def invert(z):
    return abs(z-255)
invert = np.vectorize(invert, otypes=[np.float])

def brighten(z):
    if(z<100):
        return 0.0
    if(z>120):
        return 255.0
    else:
        return z
brighten = np.vectorize(brighten, otypes=[np.float])


#test8 = cv2.imread("test2.jpg", cv2.cv2.IMREAD_GRAYSCALE)
#test8 = cv2.resize(test8, (28, 28)) 
#invert = np.vectorize(invert, otypes=[np.float])
#test8 = invert(test8)
#brighten = np.vectorize(brighten, otypes=[np.float])
#test8 = brighten(test8)
#test8 = X_test[3]
#test8=test8.astype(np.uint8)
#cv2.imwrite("/Temp/testsetnum.bmp",test8)
#reshapedimg = test8.reshape(1, 784).astype('float32')
#reshapedimg /= 255
#result = model.predict(reshapedimg)
#resultArray = np.asarray(result[0])
#resultArray.tolist().index(max(result[0]))

In [165]:
test8 = cv2.resize(test8, (28, 28)) 
cv2.imwrite("drive/afterResize.bmp",test8)

True

In [166]:
test8 = brighten(test8)
cv2.imwrite("drive/afterBrighten.bmp",test8)

True

In [167]:
X_test[1].shape == test8.shape

False

In [0]:
X_test[1]

In [85]:
X_test[1]

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0, 116, 125, 171,
        255, 255, 150,  93,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0, 169, 253, 253, 253,
        253, 253, 253, 218,  30,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0, 169, 253, 253, 253, 213,
        142, 176, 253, 253, 122,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  

In [171]:
test8 = invert(test8)
cv2.imwrite("drive/afterInvert.bmp",test8)

True

In [103]:
cv2.imwrite("drive/xtest1.bmp",X_test[1])
print(y_test[1])
cv2.imwrite("drive/xtest2.bmp",X_test[2])
print(y_test[2])
cv2.imwrite("drive/xtest3.bmp",X_test[3])
print(y_test[3])
cv2.imwrite("drive/xtest4.bmp",X_test[4])
print(y_test[4])
cv2.imwrite("drive/xtest5.bmp",X_test[5])
print(y_test[5])
cv2.imwrite("drive/xtest6.bmp",X_test[6])
print(y_test[6])

2
1
0
4
1
4


In [160]:
model = keras.models.load_model("drive/numberModel")
test_item = X_test[4].reshape(1, 784).astype('float32')
result = model.predict(test_item)
result

array([[0.01396552, 0.00236517, 0.04502108, 0.00540804, 0.59070915,
        0.02656973, 0.03823817, 0.05701685, 0.02016097, 0.20054534]],
      dtype=float32)

In [174]:
resultArray = np.asarray(result[0])
resultArray.tolist().index(max(result[0]))

8

In [172]:
test8

array([[  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 255.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0

In [173]:
reshapedimg = test8.reshape(1, 784).astype('float32')

reshapedimg /= 255
result = model.predict(reshapedimg,batch_size=1)
result
#resultArray = np.asarray(result[0])
#resultArray.tolist().index(max(result[0]))
#reshapedimg


array([[0.01302565, 0.17004663, 0.04450403, 0.12755741, 0.05662548,
        0.19996311, 0.09104463, 0.01206093, 0.23275936, 0.05241282]],
      dtype=float32)

In [99]:
y_test[4]

4

In [169]:
result

array([[0.37262315, 0.0005062 , 0.3953729 , 0.02133707, 0.01774011,
        0.08782062, 0.01360968, 0.02920977, 0.05212717, 0.00965332]],
      dtype=float32)

In [113]:
y_prob = model.predict(reshapedimg) 
y_classes = y_prob.argmax(axis=-1)
y_classes

array([5])